### import library

In [1]:
### default
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import random
import time
import copy
from datetime import datetime
from scipy.interpolate import CubicSpline

### modeling
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

### visualization
import seaborn as sns
from matplotlib import pyplot as plt

### 파라미터 세팅

In [2]:
# 센서 종류
sensors = ['C7H8', 'TMA', 'VOC', 'CO2', 'HCHO', 'H2S', 'NH3', 'CH3SH', 'SO2', 'NO2', 'CO']
cols = ['C7H8', 'TMA', 'VOC', 'CO2', 'HCHO', 'H2S', 'NH3', 'CH3SH', 'SO2', 'NO2', 'CO', 'reg_date', 'label_type']

NUM_CLASSES = 6
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

### 데이터 불러오기, 학습/검증 데이터로 구분

In [3]:
# df = pd.read_csv('df_type_total_230403.csv') # 3~7차 데이터셋
df = pd.read_csv('/home2/datapf/skim/DIC4/s.kim/df_type_total_230407.csv') # 3~9차 데이터셋
df = df[cols]

In [4]:
df_train = df[pd.to_datetime(df['reg_date']) <= datetime(2023,4,1)]
df_test = df[pd.to_datetime(df['reg_date']) > datetime(2023,4,1)]

In [5]:
len(df), len(df_train), len(df_test)

(11627, 6531, 5096)

In [6]:
df_test['label_type'].value_counts()

B     2640
ME    2456
Name: label_type, dtype: int64

### 함수정의

#### data processing / plotting

In [7]:
# 데이터셋을 만들기 위한 실험 구분
def create_experiment_number(data):
    data['lagged'] = data['reg_date'].shift(-1)
    data['reg_date'] = pd.to_datetime(data['reg_date'])
    data['lagged'] = pd.to_datetime(data['lagged'])
    data['diff'] = data['lagged'] - data['reg_date']
    data['diff'] = data['diff'] /  pd.Timedelta('1s')
    data['diff'].fillna(1,inplace=True) #마지막줄 채워주기 위함
    data.reset_index(drop=True, inplace=True)

    num = 0
    for i in range(len(data)):
        if data.loc[i,'diff'] in [1,2]:
            data.loc[i,'exp_num'] = num
        else:
            data.loc[i,'exp_num'] = num
            num+=1
    data.drop(['lagged','diff'], axis='columns', inplace=True)
    return data

# 실험 별로 시계열 데이터셋을 만들기 위한 함수
def create_windows(data, window_size=5):
    X = []
    y = []
    for num, group_df in data.groupby('exp_num'):
        data = group_df.iloc[:, 0:11].values
        labels = group_df['label_type'].values
        for i in range(len(data) - window_size + 1):
            X.append(data[i:i+window_size])
            y.append(labels[window_size-1])

    return np.array(X), np.array(y)

# 데이터셋을 train/valid/test로 나누는 함수
def split_data(X, y, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    assert train_ratio + val_ratio + test_ratio == 1, "The sum of the ratios must be equal to 1."
    assert len(X) == len(y), "The length of X and y must be the same."
    
    # numpy 배열을 torch 텐서로 변환합니다.
    X = torch.from_numpy(X)
    y = torch.from_numpy(y)

    # 데이터를 무작위로 섞기 위한 인덱스를 생성합니다.
    shuffled_indices = torch.randperm(len(X))

    # 비율에 따라 인덱스를 설정합니다.
    train_cnt = int(len(X) * train_ratio)
    val_cnt = int(len(X) * val_ratio)
    test_cnt = len(X) - (train_cnt + val_cnt)

    # 데이터를 train, validation, test 셋으로 나눕니다 (torch.index_select 사용).
    X_train, X_val, X_test = torch.index_select(X, dim=0, index=shuffled_indices).split([train_cnt, val_cnt, test_cnt], dim=0)
    y_train, y_val, y_test = torch.index_select(y, dim=0, index=shuffled_indices).split([train_cnt, val_cnt, test_cnt], dim=0)
    
    print('---'*30)
    print('Splitting Complete')
    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)
    return X_train, y_train, X_val, y_val, X_test, y_test

# torch dataloader를 만들기 위한 함수
def create_classification_dataset(data, window_size, batch_size):
    X, y = create_windows(data, window_size)
        
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

    encoder = LabelEncoder()
    y = encoder.fit_transform(y)
    y = to_categorical(y, num_classes=NUM_CLASSES)

    X = X.transpose(0,2,1)
    X = X.astype(np.float32)
    y = y.astype(np.float32)
       
    X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y)

    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)

    # DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, encoder, scaler

# torch dataloader를 만들기 위한 함수
def create_test_dataset(data, window_size, batch_size, scaler, encoder):
    X, y = create_windows(data, window_size)

    X = scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
    y = encoder.transform(y)
    y = to_categorical(y, num_classes=NUM_CLASSES)

    X = X.transpose(0,2,1)
    X = X.astype(np.float32)
    y = y.astype(np.float32)

    # numpy 배열을 torch 텐서로 변환합니다.
    X = torch.from_numpy(X)
    y = torch.from_numpy(y)

    real_test_dataset = TensorDataset(X, y)

    # DataLoader 생성
    real_test_loader = DataLoader(real_test_dataset, batch_size=batch_size, shuffle=False)

    return real_test_loader

def plot_metrics(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    acc_list = [t.item() for t in history['train_acc']]
    val_acc_list = [t.item() for t in history['val_acc']]
    plt.plot(acc_list)
    plt.plot(val_acc_list)
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='best')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history['train_loss'])
    plt.plot(history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='best')

    plt.show()

#### 학습용 데이터에 대한 증강 함수 정의

In [8]:
def add_gaussian_noise(data, mean, std_dev, noise_factor=0.1):
    noise = np.random.normal(0, std_dev * noise_factor, data.shape)
    augmented_data = data + noise
    augmented_data = np.where(augmented_data <= 0, data, augmented_data)
    return augmented_data

def scale_data_uniform_distribution(data, min_scale=0.8, max_scale=1.2):
    scaling_factor = np.random.uniform(min_scale, max_scale)
    augmented_data = data * scaling_factor
    # 방어 로직 추가
    augmented_data = np.where(augmented_data <= 0, data, augmented_data)
    return augmented_data

def scale_data_normal_distribution(data, mean_scale=1.0, std_dev_scale=0.1):
    scaling_factor = np.random.normal(mean_scale, std_dev_scale)
    augmented_data = data * scaling_factor
    # 방어 로직 추가
    augmented_data = np.where(augmented_data <= 0, data, augmented_data)
    return augmented_data

def shift_data(data, min_shift=-0.5, max_shift=0.5):
    shift_value = np.random.uniform(min_shift, max_shift)
    augmented_data = data + shift_value
    # 방어 로직 추가
    augmented_data = np.where(augmented_data <= 0, data, augmented_data)
    return augmented_data

def apply_jitter(data, mean, jitter_factor=0.05):
    jitter = np.random.uniform(-mean*jitter_factor, mean*jitter_factor, data.shape)
    augmented_data = data + jitter
    # 방어 로직 추가
    augmented_data = np.where(augmented_data <= 0, data, augmented_data)
    return augmented_data

def magnitude_warping_cubic_spline(data, num_knots=4, warping_range=0.2):
    knot_xs = np.linspace(0, 1, num_knots)
    # knot_ys = np.random.uniform(1 - warping_range, 1 + warping_range, num_knots)
    knot_ys = np.random.normal(1.0, warping_range, num_knots)
    cubic_spline = CubicSpline(knot_xs, knot_ys)
    
    data_len = len(data)
    xs = np.linspace(0, 1, data_len)
    warp_curve = cubic_spline(xs)
    
    # Reshape the warp_curve to match the data dimensions
    warp_curve = warp_curve.reshape(-1, 1)
    
    warped_data = data * warp_curve
    warped_data = np.where(warped_data <= 0, data, warped_data)
    return warped_data

def time_warping_cubic_spline(data, num_knots=4, warping_range=0.2):
    knot_xs = np.linspace(0, 1, num_knots)
    # knot_ys = np.random.uniform(1 - warping_range, 1 + warping_range, num_knots)
    knot_ys = np.random.normal(1.0, warping_range, num_knots)
    cubic_spline = CubicSpline(knot_xs, knot_ys)
    
    data_len = len(data)
    xs = np.linspace(0, 1, data_len)
    warp_curve = cubic_spline(xs)
    
    warped_xs = np.cumsum(warp_curve) / np.sum(warp_curve)
    
    # Use a loop to process each column of the data array independently
    warped_data = np.zeros_like(data)
    for i in range(data.shape[1]):
        warped_data[:, i] = np.interp(xs, warped_xs, data[:, i])
    
    # 방어 로직 추가
    warped_data = np.where(warped_data <= 0, data, warped_data)
    return warped_data

### 데이터 증강

#### 데이터 전처리(실험 구분)

In [9]:
df_train = create_experiment_number(df_train)
df_train.head()

,C7H8,TMA,VOC,CO2,HCHO,H2S,NH3,CH3SH,SO2,NO2,CO,reg_date,label_type,exp_num
0,0.000000,0.000026,0.192282,435.1554,0.00000,0.303789,0.809661,0.0,0.050841,0.012979,0.323783,2023-03-13 12:52:05,B,0.0
1,0.000000,0.000034,0.357748,431.6093,0.00000,0.528598,3.488177,0.0,0.190711,0.000000,0.291240,2023-03-13 12:52:06,B,0.0
2,0.000000,0.000045,0.498292,326.9812,0.00000,0.688187,9.506214,0.0,0.233748,0.000000,0.230189,2023-03-13 12:52:08,B,0.0
3,0.000944,0.000057,0.728273,295.0207,0.00000,0.815260,9.946881,0.0,0.176365,0.000000,0.205769,2023-03-13 12:52:09,B,0.0
4,0.002350,0.000073,1.036101,324.5227,0.79195,0.911156,9.946881,0.0,0.130409,0.000000,0.150823,2023-03-13 12:52:10,B,0.0


In [10]:
# Calculate mean and standard deviation for each sensor (column) for each class
class_stats = {}

for label in df_train['label_type'].unique():
    class_data = df_train[df_train['label_type']==label]
    class_data = class_data[sensors].values
    means = np.mean(class_data, axis=0)
    std_devs = np.std(class_data, axis=0)
    class_stats[label] = {'means': means, 'std_devs': std_devs}

In [13]:
# Apply augmentation methods
raw_data = df_train[sensors].values
labels = df_train['label_type'].values
reg_dates = df_train['reg_date'].values
exp_nums = df_train['exp_num'].values

augmented_data_1 = []
for row, label in zip(raw_data, labels):
    augmented_row = np.zeros(row.shape)
    for i, value in enumerate(row):
        aug_value = add_gaussian_noise(value, class_stats[label]['means'][i], class_stats[label]['std_devs'][i], noise_factor=0.1)
        augmented_row[i] = aug_value
    augmented_data_1.append(augmented_row)
augmented_data_1 = np.array(augmented_data_1)

augmented_data_2 = []
for row, label in zip(raw_data, labels):
    augmented_row = np.zeros(row.shape)
    for i, value in enumerate(row):
        aug_value = scale_data_uniform_distribution(value)
        augmented_row[i] = aug_value
    augmented_data_2.append(augmented_row)
augmented_data_2 = np.array(augmented_data_2)

augmented_data_3 = []
for row, label in zip(raw_data, labels):
    augmented_row = np.zeros(row.shape)
    for i, value in enumerate(row):
        aug_value = scale_data_normal_distribution(value)
        augmented_row[i] = aug_value
    augmented_data_3.append(augmented_row)
augmented_data_3 = np.array(augmented_data_3)

# augmented_data_4 = []
# for row, label in zip(raw_data, labels):
#     augmented_row = np.zeros(row.shape)
#     for i, value in enumerate(row):
#         aug_value = shift_data(value)
#         augmented_row[i] = aug_value
#     augmented_data_4.append(augmented_row)
# augmented_data_4 = np.array(augmented_data_4)

augmented_data_5 = []
for row, label in zip(raw_data, labels):
    augmented_row = np.zeros(row.shape)
    for i, value in enumerate(row):
        aug_value = apply_jitter(value, class_stats[label]['means'][i] )
        augmented_row[i] = aug_value
    augmented_data_5.append(augmented_row)
augmented_data_5 = np.array(augmented_data_5)

augmented_data_6 = []
augmented_data_7 = []
for exp in df_train['exp_num'].unique():
    each = df_train[df_train['exp_num']==exp]
    each = each[sensors].values
    augmented_data_6.append(magnitude_warping_cubic_spline(each))
    augmented_data_7.append(time_warping_cubic_spline(each))

# convert (120, ) ==> (6531, 11)
augmented_data_6 = np.vstack(augmented_data_6) 
augmented_data_7 = np.vstack(augmented_data_7)

augmented_data_c1 = []
for row, label in zip(raw_data, labels):
    augmented_row = np.zeros(row.shape)
    for i, value in enumerate(row):
        aug_value = add_gaussian_noise(value, class_stats[label]['means'][i], class_stats[label]['std_devs'][i], noise_factor=0.1)
        aug_value = scale_data_uniform_distribution(aug_value)
        aug_value = apply_jitter(aug_value, class_stats[label]['means'][i])
        augmented_row[i] = aug_value
    augmented_data_c1.append(augmented_row)
augmented_data_c1 = np.array(augmented_data_c1)
augmented_data_c1 = pd.DataFrame(augmented_data_c1, columns=sensors)
augmented_data_c1['exp_num'] = exp_nums

augmented_data_8 = []
augmented_data_9 = []
for exp in augmented_data_c1['exp_num'].unique():
    each = augmented_data_c1[augmented_data_c1['exp_num']==exp]
    each = each[sensors].values
    augmented_data_8.append(magnitude_warping_cubic_spline(each))
    augmented_data_9.append(time_warping_cubic_spline(each))

# convert (120, ) ==> (6531, 11)
augmented_data_8 = np.vstack(augmented_data_8) 
augmented_data_9 = np.vstack(augmented_data_9)

# Save augmented data to a new DataFrame
augmented_df_1 = pd.DataFrame(augmented_data_1, columns=sensors)
augmented_df_1['label_type'] = labels
augmented_df_1['reg_date'] = reg_dates
augmented_df_1['exp_num'] = exp_nums
augmented_df_1['aug_type'] = 'gaussian_noise'
augmented_df_2 = pd.DataFrame(augmented_data_2, columns=sensors)
augmented_df_2['label_type'] = labels
augmented_df_2['reg_date'] = reg_dates
augmented_df_2['exp_num'] = exp_nums
augmented_df_2['aug_type'] = 'scaling(uniform)'
augmented_df_3 = pd.DataFrame(augmented_data_3, columns=sensors)
augmented_df_3['label_type'] = labels
augmented_df_3['reg_date'] = reg_dates
augmented_df_3['exp_num'] = exp_nums
augmented_df_3['aug_type'] = 'scaling(normal)'
# augmented_df_4 = pd.DataFrame(augmented_data_4, columns=sensors)
# augmented_df_4['label_type'] = labels
# augmented_df_4['reg_date'] = reg_dates
# augmented_df_4['exp_num'] = exp_nums
augmented_df_5 = pd.DataFrame(augmented_data_5, columns=sensors)
augmented_df_5['label_type'] = labels
augmented_df_5['reg_date'] = reg_dates
augmented_df_5['exp_num'] = exp_nums
augmented_df_5['aug_type'] = 'jittering'
augmented_df_6 = pd.DataFrame(augmented_data_6, columns=sensors)
augmented_df_6['label_type'] = labels
augmented_df_6['reg_date'] = reg_dates
augmented_df_6['exp_num'] = exp_nums
augmented_df_6['aug_type'] = 'magnitude_warping'
augmented_df_7 = pd.DataFrame(augmented_data_7, columns=sensors)
augmented_df_7['label_type'] = labels
augmented_df_7['reg_date'] = reg_dates
augmented_df_7['exp_num'] = exp_nums
augmented_df_7['aug_type'] = 'time_warping'
augmented_df_8 = pd.DataFrame(augmented_data_8, columns=sensors)
augmented_df_8['label_type'] = labels
augmented_df_8['reg_date'] = reg_dates
augmented_df_8['exp_num'] = exp_nums
augmented_df_8['aug_type'] = 'combination1'
augmented_df_9 = pd.DataFrame(augmented_data_9, columns=sensors)
augmented_df_9['label_type'] = labels
augmented_df_9['reg_date'] = reg_dates
augmented_df_9['exp_num'] = exp_nums
augmented_df_9['aug_type'] = 'combination2'

augmented_df = pd.concat([augmented_df_1, augmented_df_2, augmented_df_3
# , augmented_df_4
, augmented_df_5, augmented_df_6, augmented_df_7, augmented_df_8, augmented_df_9])
augmented_df.reset_index(drop=True, inplace=True)
augmented_df['is_aug'] = 1

In [14]:
df_train['is_aug'] = 0
df_train['aug_type'] = 'original'
df_train_total = pd.concat([df_train,augmented_df])

In [15]:
df_train_total['is_aug'].value_counts()

1    52248
0     6531
Name: is_aug, dtype: int64

In [16]:
df_train_total.groupby(['is_aug','label_type']).mean()

C7H8       TMA       VOC          CO2      HCHO  \
is_aug label_type                                                        
0      A           0.009780  0.000033  0.315670  3282.466437  0.000512   
       B           0.062161  0.000261  3.786709   280.845259  9.511448   
       C           0.003430  0.000031  0.275784  2549.303650  0.000227   
       EN          0.026686  0.000040  0.482237    39.275725  0.007571   
       ME          0.036099  0.000114  0.794137   192.520571  0.775849   
       MO          0.016899  0.000060  0.768089    78.141789  0.006571   
1      A           0.009763  0.000033  0.316687  3301.816065  0.000804   
       B           0.062132  0.000260  3.792596   283.393924  9.507166   
       C           0.003458  0.000030  0.274683  2510.382076  0.000349   
       EN          0.026626  0.000040  0.482268    42.043345  0.009970   
       ME          0.036130  0.000114  0.790923   195.864418  0.785160   
       MO          0.016951  0.000060  0.766739    80.458056  0.007730   

                        H2S       NH3     CH3SH       SO2       NO2        CO  \
is_aug label_type                                                               
0      A           0.888562  2.129120  0.009476  0.024296  0.011137  0.574519   
       B           0.894038  9.492534  0.000244  0.007592  0.012404  0.588091   
       C           0.939731  2.102958  0.010678  0.035658  0.006109  0.274356   
       EN          0.805855  4.409067  0.002131  0.014499  0.023759  0.398367   
       ME          1.014770  6.617821  0.073342  0.006931  0.007405  0.696040   
       MO          0.811076  8.925683  0.006587  0.010757  0.021719  0.542908   
1      A           0.887493  2.119665  0.009723  0.024466  0.011981  0.580672   
       B           0.896619  9.522685  0.000290  0.007958  0.013986  0.592828   
       C           0.928356  2.070617  0.010938  0.035648  0.006714  0.275686   
       EN          0.802425  4.392820  0.002164  0.014741  0.025362  0.403486   
       ME          1.011180  6.571229  0.075791  0.007253  0.008258  0.699077   
       MO          0.809514  8.896214  0.006614  0.011022  0.023582  0.540151   

                     exp_num  
is_aug label_type             
0      A           76.753223  
       B           56.766493  
       C           77.087737  
       EN          66.741490  
       ME          56.117647  
       MO          62.390223  
1      A           76.753223  
       B           56.766493  
       C           77.087737  
       EN          66.741490  
       ME          56.117647  
       MO          62.390223

### pytorch를 사용한 딥러닝 모델 구현

In [17]:
# Hyperparameter setting
batch_size = 32
num_classes = NUM_CLASSES
num_epochs = 50
window_size = 7
feature_len = 11

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Detect if we have a GPU available

In [18]:
# seed 고정
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [19]:
# Dataloader 구축
# data shape: (batch_size x input_size x seq_len) => (32, 11, 5)
train_loader, valid_loader, test_loader, encoder, scaler = create_classification_dataset(df_train_total, window_size, batch_size)

------------------------------------------------------------------------------------------
Splitting Complete
torch.Size([46447, 11, 7]) torch.Size([46447, 6]) torch.Size([5805, 11, 7]) torch.Size([5805, 6]) torch.Size([5807, 11, 7]) torch.Size([5807, 6])


In [20]:
# DataLoader에서 데이터 형태 확인
for batch_idx, (inputs, targets) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1} - Inputs shape: {inputs.shape}, Targets shape: {targets.shape}")
    break

Batch 1 - Inputs shape: torch.Size([32, 11, 7]), Targets shape: torch.Size([32, 6])


In [21]:
# DataLoader에서 데이터 형태 확인
for batch_idx, (inputs, targets) in enumerate(test_loader):
    print(f"Batch {batch_idx + 1} - Inputs shape: {inputs.shape}, Targets shape: {targets.shape}")
    break

Batch 1 - Inputs shape: torch.Size([32, 11, 7]), Targets shape: torch.Size([32, 6])


In [26]:
# 1-dimensional convolution layer로 구성된 CNN 모델
# 2개의 1-dimensional convolution layer와 1개의 fully-connected layer로 구성되어 있음
class CNN_1D(nn.Module):
    def __init__(self, in_channel, out_channel, num_classes, dropout_rate=0.2):

        self.in_channel = in_channel
        self.out_channel = out_channel
        self.num_classes = num_classes

        super(CNN_1D, self).__init__()
        # 첫 번째 1-dimensional convolution layer 구축
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channel, out_channel, kernel_size=3, padding=1),
            nn.ReLU(),
#             nn.AvgPool1d(kernel_size=2, stride=2),
            # nn.MaxPool1d(kernel_size=2, stride=2)
            nn.MaxPool1d(kernel_size=2, stride=2, padding=0),
            nn.Dropout(dropout_rate)
        )
        # 두 번째 1-dimensional convolution layer 구축
        self.layer2 = nn.Sequential(
            nn.Conv1d(out_channel, 32, kernel_size=3, padding=1),
            nn.ReLU(),
#             nn.AvgPool1d(kernel_size=2, stride=2),
            # nn.MaxPool1d(kernel_size=2, stride=2)
            nn.MaxPool1d(kernel_size=2, stride=2, padding=0),
            nn.Dropout(dropout_rate)
        )
        # fully-connected layer 구축
        self.fc = nn.Linear(32*1, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [27]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_rate=0.2, bidirectional=False, rnn_type='rnn'):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn_type = rnn_type.lower()
        self.num_directions = 2 if bidirectional == True else 1
        self.dropout_rate = dropout_rate
        
        if self.rnn_type == 'rnn':
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate, bidirectional=bidirectional)
        elif self.rnn_type == 'lstm':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate, bidirectional=bidirectional)
        elif self.rnn_type == 'gru':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate, bidirectional=bidirectional)
        else:
            raise ValueError(f"Invalid rnn_type: {rnn_type}, expected one of ['rnn', 'lstm', 'gru']")
        
        # bidirectional에 따른 fc layer 구축
        # bidirectional 여부에 따라 hidden state의 shape가 달라짐 (True: 2 * hidden_size, False: hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(self.num_directions * hidden_size, num_classes)

    def forward(self, x):
        # data dimension: (batch_size x input_size x seq_len) -> (batch_size x seq_len x input_size)로 변환
        x = torch.transpose(x, 1, 2) # x.permute(0, 2, 1) 와 결과적으로는 동일
        rnn_out, hidden = self.rnn(x)

        # LSTMs, the hidden state is a tuple containing both the hidden state and cell state. 
        if self.rnn_type == 'lstm':
            hidden_state, _ = hidden
        else:
            hidden_state = hidden

        if bidirectional:
            forward_out = rnn_out[:, -1, :self.hidden_size]
            backward_out = rnn_out[:, 0, self.hidden_size:]
            last_time_step_out = torch.cat((forward_out, backward_out), dim=1)
        else:
            last_time_step_out = rnn_out[:, -1, :]
        
        dropped_out = self.dropout(last_time_step_out)
        out = self.fc(dropped_out)
        
        return out

In [28]:
import torch
import torch.nn as nn

class CRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.2, kernel_size=3, num_layers=1):
        super(CRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=32, kernel_size=kernel_size, stride=1, padding=1),
            nn.BatchNorm1d(num_features=32),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=kernel_size, stride=1, padding=1),
            nn.BatchNorm1d(num_features=64),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=kernel_size, stride=1, padding=1),
            nn.BatchNorm1d(num_features=128),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=kernel_size, stride=1, padding=1),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
        )
        self.dropout_cnn_rnn = nn.Dropout(dropout_rate)
        self.rnn = nn.GRU(input_size=256, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=(0 if num_layers == 1 else dropout_rate))
        self.dropout_rnn_fc = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.cnn(x)
        x = x.permute(0, 2, 1)  # (batch_size, window_size, feature_size)
        x = self.dropout_cnn_rnn(x)
        output, hidden = self.rnn(x)
        output = self.dropout_rnn_fc(output[:, -1, :])
        output = self.fc(output)
        return output

In [29]:
# 1D CNN 구축
cnn_default = CNN_1D(in_channel=feature_len, out_channel=16, num_classes=NUM_CLASSES, dropout_rate=0) # in_channel은 feature의 갯수
cnn_default = cnn_default.to(device)
print(cnn_default)

CNN_1D(
  (layer1): Sequential(
    (0): Conv1d(11, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0, inplace=False)
  )
  (fc): Linear(in_features=32, out_features=6, bias=True)
)


In [30]:
# 1D CNN with dropout 구축
cnn_dropout = CNN_1D(in_channel=feature_len, out_channel=16, num_classes=NUM_CLASSES, dropout_rate=0.25) # in_channel은 feature의 갯수
cnn_dropout = cnn_dropout.to(device)
print(cnn_dropout)

CNN_1D(
  (layer1): Sequential(
    (0): Conv1d(11, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
  )
  (fc): Linear(in_features=32, out_features=6, bias=True)
)


In [31]:
def train_model(model, dataloaders, criterion, num_epochs, optimizer):
    since = time.time()

    train_acc_history = []
    val_acc_history = []
    train_loss_history = []
    val_loss_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        train_loss = 0
        val_loss = 0
        train_corrects = 0
        val_corrects = 0
        train_total = 0
        val_total = 0

        # 각 epoch마다 순서대로 training과 validation을 진행
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 training mode로 설정
            else:
                model.eval()   # 모델을 validation mode로 설정            

            # training과 validation 단계에 맞는 dataloader에 대하여 학습/검증 진행
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # parameter gradients를 0으로 설정
                optimizer.zero_grad()

                # forward
                # training 단계에서만 gradient 업데이트 수행
                with torch.set_grad_enabled(phase == 'train'):
                    # input을 model에 넣어 output을 도출한 후, loss를 계산함
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # output 중 최댓값의 위치에 해당하는 class로 예측을 수행
                    _, preds = torch.max(outputs, 1)

                    # backward (optimize): training 단계에서만 수행
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # batch별 loss를 축적함
                if phase == 'train':
                    # running_loss += loss.item() * inputs.size(0)
                    # running_corrects += torch.sum(preds == torch.argmax(labels, dim=-1))
                    # running_total += labels.size(0)

                    train_loss += loss.item() * inputs.size(0)
                    # train_loss += loss.item() 
                    train_corrects += torch.sum(preds == torch.argmax(labels, dim=-1))
                    train_total += labels.size(0)

                else:
                    val_loss += loss.item() * inputs.size(0)
                    # val_loss += loss.item()
                    val_corrects += torch.sum(preds == torch.argmax(labels, dim=-1))
                    val_total += labels.size(0)
            
        train_loss_epoch = train_loss/train_total
        val_loss_epoch = val_loss/val_total
        train_acc_epoch = train_corrects.double() /train_total
        val_acc_epoch = val_corrects.double() /val_total

        # epoch의 loss 및 accuracy 도출
        # epoch_loss = running_loss / running_total
        # epoch_acc = running_corrects.double() / running_total

        # validation 단계에서 validation loss가 감소할 때마다 best model 가중치를 업데이트함
        if phase == 'val' and val_acc_epoch > best_acc:
            best_acc = val_acc_epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            print('update model parameter')

        train_acc_history.append(train_acc_epoch)
        val_acc_history.append(val_acc_epoch)
        train_loss_history.append(train_loss_epoch)
        val_loss_history.append(val_loss_epoch)
            
            # if phase == 'val':
            #     val_acc_history.append(val_acc_epoch)
            
        print(f'{epoch+1} epoch | Train loss: {train_loss_epoch:.3f}, Valid loss: {val_loss_epoch:.3f}, Train acc: {train_acc_epoch:.3f}, Valid acc: {val_acc_epoch:.3f}')
        # print(f'{epoch+1} epoch | Train acc: {train_acc_epoch:.3f}, Valid acc: {val_acc_epoch:.3f}')
        print()

    # 전체 학습 시간 계산
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # validation loss가 가장 낮았을 때의 best model 가중치를 불러와 best model을 구축함
    model.load_state_dict(best_model_wts)
    history = {'train_acc':train_acc_history, 'val_acc':val_acc_history, 'train_loss':train_loss_history, 'val_loss':val_loss_history}
    
    # best model 가중치 저장
    # torch.save(best_model_wts, '../output/best_model.pt')
    return model, history

In [32]:
# trining 단계에서 사용할 Dataloader dictionary 생성
dataloaders_dict = {
    'train': train_loader,
    'val': valid_loader
}
# loss function 설정
criterion = nn.CrossEntropyLoss()

In [ ]:
# 모델 학습
cnn_default_model, history = train_model(cnn_default, dataloaders_dict, criterion, num_epochs, optimizer=optim.Adam(cnn_default.parameters(), lr=0.001))

Epoch 1/50
----------
update model parameter
1 epoch | Train loss: 0.665, Valid loss: 0.432, Train acc: 0.728, Valid acc: 0.836

Epoch 2/50
----------
update model parameter
2 epoch | Train loss: 0.372, Valid loss: 0.325, Train acc: 0.853, Valid acc: 0.869

Epoch 3/50
----------
update model parameter
3 epoch | Train loss: 0.292, Valid loss: 0.269, Train acc: 0.884, Valid acc: 0.903

Epoch 4/50
----------
4 epoch | Train loss: 0.245, Valid loss: 0.248, Train acc: 0.902, Valid acc: 0.892

Epoch 5/50
----------
update model parameter
5 epoch | Train loss: 0.216, Valid loss: 0.209, Train acc: 0.915, Valid acc: 0.922

Epoch 6/50
----------
update model parameter
6 epoch | Train loss: 0.196, Valid loss: 0.196, Train acc: 0.923, Valid acc: 0.923

Epoch 7/50
----------
7 epoch | Train loss: 0.179, Valid loss: 0.206, Train acc: 0.930, Valid acc: 0.902

Epoch 8/50
----------
update model parameter
8 epoch | Train loss: 0.165, Valid loss: 0.161, Train acc: 0.936, Valid acc: 0.939

Epoch 9/50
---

In [ ]:
# 모델 학습
cnn_dropout_model, cnn_dropout_history = train_model(cnn_dropout, dataloaders_dict, criterion, num_epochs, optimizer=optim.Adam(cnn_dropout.parameters(), lr=0.001))

In [ ]:
def test_model(model, test_loader):
    model.eval()   # 모델을 validation mode로 설정

    pred_list = []
    true_list = []

    # test_loader에 대하여 검증 진행 (gradient update 방지)
    with torch.no_grad():
        corrects = 0
        total = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # forward
            # input을 model에 넣어 output을 도출
            outputs = model(inputs)

            # output 중 최댓값의 위치에 해당하는 class로 예측을 수행
            _, preds = torch.max(outputs, 1)

            # batch별 정답 개수를 축적함
            corrects += torch.sum(preds == torch.argmax(labels, dim=-1))
            total += labels.size(0)

            pred_list.extend(preds.detach().numpy())
            true_list.extend(torch.argmax(labels, dim=-1).detach().numpy())

    # accuracy를 도출함
    test_acc = corrects.double() / total
    print('Testing Acc: {:.4f}'.format(test_acc))

    return pred_list, true_list

In [ ]:
# 모델 검증 
pred_list, true_list = test_model(model, test_loader)

In [ ]:
# 평가지표 계산
test_acc = accuracy_score(true_list, pred_list)
test_rec = recall_score(true_list, pred_list, average='macro')
test_prec = precision_score(true_list, pred_list, average='macro')
test_f1 = f1_score(true_list, pred_list, average='macro')

print('Test Accuracy   : {:.3f}'.format(test_acc))
print('Test Sensitivity: {:.3f}'.format(test_rec))
print('Test Precision  : {:.3f}'.format(test_prec))
print('Test F1 Score   : {:.3f}'.format(test_f1))

In [ ]:
plot_metrics(history)

In [ ]:
# confusion matrix 계산
cm = confusion_matrix(y_true=true_list, y_pred=pred_list)
cm_df = pd.DataFrame(cm)

cm_df.index = encoder.classes_
cm_df.columns = encoder.classes_

# confusion matrix 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(data=cm_df, annot=True, fmt='d', annot_kws={'size': 15}, cmap='Blues')
plt.xlabel('Predicted', size=20)
plt.ylabel('True', size=20)
plt.show()

In [ ]:
df_test = create_experiment_number(df_test)
df_test.head()

In [ ]:
# Test Dataloader 구축
# data shape: (batch_size x input_size x seq_len) => (32, 11, 5)
real_test_dataloader = create_test_dataset(df_test, window_size, batch_size, encoder=encoder, scaler=scaler)

In [ ]:
r_pred_list, r_true_list = test_model(model, real_test_dataloader)

In [ ]:
encoding_numbers = list(range(0,len(encoder.classes_)))
label_dict = dict(zip(encoding_numbers, encoder.inverse_transform(encoding_numbers)))
print(label_dict)

In [ ]:
# confusion matrix 계산
cm = confusion_matrix(y_true=r_true_list, y_pred=r_pred_list)
cm_df = pd.DataFrame(cm, columns=np.unique(r_pred_list), index=np.unique(r_pred_list))

new_index = [label_dict[i] for i in cm_df.index]
new_columns = [label_dict[i] for i in cm_df.columns]

# 인덱스와 컬럼 이름 변경
cm_df = cm_df.rename(index=dict(zip(cm_df.index, new_index)), columns=dict(zip(cm_df.columns, new_columns)))

# confusion matrix 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(data=cm_df, annot=True, fmt='d', annot_kws={'size': 15}, cmap='Blues')
plt.xlabel('Predicted', size=20)
plt.ylabel('True', size=20)
plt.show()